本notebook主要展示了cnn的示例
- 主要参考了morvanzhou的博客[参考](https://morvanzhou.github.io/tutorials/machine-learning/keras/2-3-CNN/)
- 在windows上使用了anaconda+keras+jupyter notebook实现

In [1]:
import pandas as pd
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.utils import np_utils
from keras.datasets import mnist
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
np.random.seed(1337)

Using TensorFlow backend.


## 导入数据

In [2]:
dir = r'/Users/leebin/proj/py_collection/py-tut/kaggle-exe/digital-recongnizer/data/'
train_path = dir +  r'train.csv'
test_path =  dir + r'test.csv'
submit_path = dir + r'sample_submission.csv'

In [3]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
submit_data = pd.read_csv(submit_path)

## 查看维度

In [4]:
train_data.shape

(42000, 785)

In [5]:
test_data.shape

(28000, 784)

In [6]:
train_data.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

## 归一化

In [7]:
# 判断是不是灰度图
max(train_data[max(train_data)])

255

In [8]:
# test不包含标签，先需要归一化处理
test_data = test_data/255.0

In [9]:
max(train_data[max(train_data)])

255

## 得到训练数据

In [10]:
X_train = train_data.drop(['label'], axis=1)
y_train = train_data['label']
X_test = test_data

In [11]:
# train去掉标签后,需要归一化处理
X_train = X_train / 255.0

In [12]:
X_train.columns

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [13]:
y_train

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [14]:
max(X_test[max(X_test)])

1.0

In [15]:
X_train

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X_train = X_train.values.reshape(-1, 1, 28, 28)
X_test = X_test.values.reshape(-1, 1, 28, 28)

In [17]:
y_train = np_utils.to_categorical(y_train, num_classes=10)

## 建模

In [18]:
model = Sequential()

In [19]:
model.add(Convolution2D(
    batch_input_shape=(None, 1, 28, 28),
    filters=32,
    kernel_size=5,
    strides=1,
    padding='same',
    data_format='channels_first',
))

In [20]:
model.add(MaxPooling2D(
    pool_size=2,
    strides=2,
    padding='same',
    data_format='channels_first',
))

In [21]:
model.add(Convolution2D(
    64, 5, strides=1,
    padding='same', data_format='channels_first'))
model.add(Activation('relu'))

In [22]:
model.add(MaxPooling2D(
    2, 2, 'same', data_format='channels_first'
))

In [23]:
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))

In [24]:
model.add(Dense(10))
model.add(Activation('softmax'))

In [25]:
adam = Adam(lr=1e-4)

In [26]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1, batch_size=100)

Epoch 1/1


In [ ]:
r = model.predict_classes(X_test)

In [ ]:
r

In [ ]:
submissions = pd.DataFrame({
    "ImageId": list(range(1, len(r)+1)),
    "Label": r
})

In [ ]:
submissions

In [ ]:
submissions.to_csv("DR.csv", index=False, header=True)